## Normalizare textul simptomelor:
   - transformă toate valorile în **șiruri de caractere**,
   - elimină **spațiile goale** de la început/sfârșit,
   - convertește la **litere mici** pentru a evita duplicatele de tipul `"Itching"` vs `"itching"`.

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import xgboost as xgb

df = pd.read_csv("data/DiseaseAndSymptoms.csv")

symptom_cols = [c for c in df.columns if c.startswith("Symptom_")]

for col in symptom_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()

all_symptoms = sorted(set(df[symptom_cols].stack().dropna().unique()) - {''})

print(f"Total unique symptoms found: {len(all_symptoms)}")

Total unique symptoms found: 132


## Transofrmare date:
   - Creez pentru fiecare simptom unic o coloană binară (1 dacă simptomul este prezent în rândul respectiv, 0 dacă lipsește) — proces numit multi-hot encoding.
   - Definesc matricea de trăsături X, care conține aceste coloane binare pentru toate simptomele.
   - Extrage coloana Disease drept variabilă țintă y.
   - Convertește numele bolilor din y în valori numerice (y_encoded) folosind LabelEncoder, pentru ca modelul XGBoost să poată lucra cu ele.

In [2]:
# Convert each row into a binary vector (multi-hot encoding)
for symptom in all_symptoms:
    df[symptom] = df[symptom_cols].apply(lambda x: int(symptom in x.values), axis=1)

# Define features (X) and target (y)
X = df[all_symptoms]

y = df['Disease']

# 7️⃣ Encode disease labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(y_encoded)

/var/folders/fw/wwhdmw591v3570fxtz50q1l40000gn/T/ipykernel_96452/471652356.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[symptom] = df[symptom_cols].apply(lambda x: int(symptom in x.values), axis=1)
/var/folders/fw/wwhdmw591v3570fxtz50q1l40000gn/T/ipykernel_96452/471652356.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[symptom] = df[symptom_cols].apply(lambda x: int(symptom in x.values), axis=1)
/var/folders/fw/wwhdmw591v3570fxtz50q1l40000gn/T/ipykernel_96452/471652356.py:3: PerformanceWarning: DataFrame is high

[15 15 15 ... 38 35 27]


/var/folders/fw/wwhdmw591v3570fxtz50q1l40000gn/T/ipykernel_96452/471652356.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[symptom] = df[symptom_cols].apply(lambda x: int(symptom in x.values), axis=1)
/var/folders/fw/wwhdmw591v3570fxtz50q1l40000gn/T/ipykernel_96452/471652356.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[symptom] = df[symptom_cols].apply(lambda x: int(symptom in x.values), axis=1)
/var/folders/fw/wwhdmw591v3570fxtz50q1l40000gn/T/ipykernel_96452/471652356.py:3: PerformanceWarning: DataFrame is high

## Impartire date
   - 70% antrenament
   - 30% test

In [3]:
# 8️⃣ Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

## Model
   - Initializarea modelului cu hiperparametrii
   - Antrenarea modelului cu datele prelucrate

In [4]:
# 9️⃣ Initialize and train the XGBoost classifier
model = xgb.XGBClassifier(
    n_estimators=100,          
    learning_rate=0.1,         
    max_depth=3,               
    subsample=0.7,             
    colsample_bytree=0.7,      
    reg_lambda=5,              
    reg_alpha=1,               
    eval_metric='mlogloss',
    random_state=42
)

print("Training the model...")
model.fit(X_train, y_train)

Training the model...


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


## Evalurare
   - acuratete :  modelul intoarce o acuratete de 100%, un evident overfiting din cauza setului de date
      - posibile imbunatatiri: modificarea hiperparametrilor sau a modului de antrenare
   - matrice de confuzie

In [5]:
# 🔟 Evaluate the model
print("\nEvaluating...")
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Accuracy: {acc * 100:.2f}%\n")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

#confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=label_encoder.classes_, columns=label_encoder.classes_)
print("\nConfusion Matrix:\n", cm_df)


Evaluating...

✅ Accuracy: 100.00%

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        48
                                   AIDS       1.00      1.00      1.00        48
                                   Acne       1.00      1.00      1.00        48
                    Alcoholic hepatitis       1.00      1.00      1.00        48
                                Allergy       1.00      1.00      1.00        48
                              Arthritis       1.00      1.00      1.00        48
                       Bronchial Asthma       1.00      1.00      1.00        48
                   Cervical spondylosis       1.00      1.00      1.00        48
                            Chicken pox       1.00      1.00      1.00        48
                    Chronic cholestasis       1.00      1.00      1.00        48
                            Common Cold       1.00      1.00      1.00 

## Exemplu de predictie

In [6]:
def predict_disease(symptoms_list):
    """Predict disease from a list of symptom strings"""
    input_data = np.zeros(len(all_symptoms))
    for s in symptoms_list:
        s_clean = s.strip().lower()
        if s_clean in all_symptoms:
            input_data[all_symptoms.index(s_clean)] = 1

    d_pred = model.predict([input_data])[0]
    disease = label_encoder.inverse_transform([d_pred])[0]
    probs = model.predict_proba([input_data])[0]
    confidence = np.max(probs) * 100

    print("\nPredicted Disease:", disease)
    print(f"Confidence: {confidence:.1f}%")

    # Show top-3 likely diseases
    top3_idx = np.argsort(probs)[::-1][:3]
    print("\nTop 3 Predictions:")
    for idx in top3_idx:
        print(f" - {label_encoder.inverse_transform([idx])[0]} ({probs[idx]*100:.1f}%)")

predict_disease(["itching", "skin_rash", "dischromic _patches"])



Predicted Disease: Fungal infection
Confidence: 77.4%

Top 3 Predictions:
 - Fungal infection (77.4%)
 - Drug Reaction (1.3%)
 - Acne (1.0%)


## Salvare model pentru integrare

In [8]:
# 💾 Save model, encoder, and symptom list
model.save_model("xgb_model.json")

import joblib
np.save("utils/symptoms_list.npy", np.array(all_symptoms))
joblib.dump(label_encoder, "utils/label_encoder.pkl")

print("Model, symptoms, and encoder saved successfully!")


Model, symptoms, and encoder saved successfully!
